In [78]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [7]:
#try to create a new folder 'data'
try:
    os.mkdir('data')
    print("directory created")
except Exception as e: #'e' means for the same instruction - in this case, the directory 'data'
    print(f"Directory already exists {e}")
    pass

Directory already exists [WinError 183] Cannot create a file when that file already exists: 'data'


In [27]:
def save(t,n):
    #input the job page content as soup object t
    #convert t to string s
    s=t.prettify()
    
    #save the job page content in a text file number n as string
    with open(f"data/{n}.txt","w",encoding='utf-8') as g:
        g.write(s)
    return

In [28]:
def scrape(x,y):
    #to scrape the pages from page x to page y where y>=x.
    #it returns a list of soup objects of job page content
    
    #get the link
    URL = "https://hk.jobsdb.com/hk/search-jobs/data-scientist/"
    
    for p in range(x,y+1):
        #to make valid URL e.g https://hk.jobsdb.com/hk/search-jobs/data-science/1
        nextlink = URL+str(p)
        
        #if the link exists,get the resources
        if nextlink:
            html = requests.get(nextlink)
            soup = BeautifulSoup(html.text, "html.parser")

            #to find the links(directing to the job pages) from the resources
            #list l stores all the links
            l=[]
            for i in soup.find_all('a', href=True):
                if ('hk/en/job' in i['href'])&(i['href'][0]=="/"):
                    l.append("https://hk.jobsdb.com"+i['href'])

            #go to each job page by each link and get the content
            #result (list) stores the job contents
            result=[]
            for k in range(len(l)):
                i=l[k]
                #print("going to",i) #for checking only
                subhtml = requests.get(i)
                #print("success or not?",subhtml) #for checking only
                jobcontent = BeautifulSoup(subhtml.text, "html.parser")
                result.append(jobcontent)
                save(jobcontent,(p-1)*30+k+1)
                
        
        #if the link does not exist, it breaks the for loop.
        #i found that if the link does not exist, the DB website will
          #redirect you to page 1 automatically, which makes this part of code
          #does not work.
        else:
            print("Page",p,"does not exist.")
            break
    return result

In [29]:
#scrape the pages from 133 to 171
#scrape(133,171) we have done it.

section I: scrapping and saving as text
_______________________________________________
section II: gathering data

In [43]:
#to read the job content.txt:
def read(casenum):
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        text=onlyread.read()
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        line=onlyread.readlines()
    soup = BeautifulSoup(text, "html.parser")
    return text,line,soup

In [72]:
#transferred
def wordlocation(text,line,word,min_line,max_line):
    #it returns all the non-coding sentence(str) in a list after the target word
    #if max=min, a string is returned (but not a list)
    
    #text,line: pass variable text and line to the function
    #word: input the target word
    #min/max: the range of the non-coding sentence after the target word
    
    s=str(re.search(word,text))
    l=re.findall('\d+', s)
    if l==[]:
        #print("the word does not exist, return none") #for checking only
        return
    g=text[:int(l[1])+1].count("\n")

    p=1
    result=[]
    for j in range(g+1,g+60):
        if line[j].strip()[0]!="<":
            if ("Additional Information" in line[j].strip())==True:
                #print("already in add. info part") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if line[j].strip()=="N/A":
                #print("they wrote N/A") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if (p<=max_line)&(p>=min_line)&(len(line[j].strip())<800):
                if max_line==min_line:
                    return line[j].strip()
                else:
                    result.append(line[j].strip())
                    if p==max_line:
                        return result    
            p+=1

    #print("end of search. return none if nth is found. return a list if sth is found.") #for checking only
    if max_line==min_line:
        return
    else:
        if result==[]:
            return
        else:
            return result

In [45]:
#find post date. It returns date(str) in dd-mmm-yy (date,month,year format)
def date(text):
    s=str(re.search("Posted on",text))
    if s==None:
        s=str(re.search("Posted",text))
        if s==None:
            return
        else:
            return "05-Nov-21"
    else:
        datelist=re.findall('\d+', s)
        y=text[int(datelist[1])+1:int(datelist[1])+10].strip()
        if y[1]=="-":
            return '0'+y
        else:
            return y 

In [46]:
#find job title. It returns a string
def job(soup):
    return soup.find('h1').text.strip()

In [47]:
#it returns company name in string
def company(soup):
    longclass="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc2 _1d0g9qk4 _18qlyvc9"
    return soup.find('span',class_=longclass).find_next('span',class_=longclass).text.strip()

In [62]:
#find location in string
def location(soup):
    y = soup.find('div',class_="sx2jih0 zcydq832").text
    if y==None:
        return
    else:
        return y.strip()

In [49]:
#find industry in string
def industry(text,line):
    s=str(re.search("Additional Company Information\n",text))
    l=re.findall('\d+', s)
    g=text[:int(l[1])+1].count("\n")
    
    p=0
    for j in range(g,g+20):
        if line[j].strip()[0]!="<":
            p+=1
        if p==2:
            return line[j].strip()
    #print("no industry found. return none") #for checking only
    return

In [50]:
#find years of experience
def exp(text,line):
    return wordlocation(text,line,"Years of Experience\n",1,1)

In [51]:
#find requirements/responsibility in list
def req(text,line,word):
    return wordlocation(text,line,word,1,20)

In [227]:
#find job description
def descri(text,line):
    return wordlocation(text,line,"Job Description",1,80)

In [ ]:
###summary of functions:
#read(casenum): return text,line,soup
#date(text): return str(dd-mmm-yy) / none
#job(soup): return string
#company(soup): return string
#location(soup): return string / none
#industry(text,line): return string / none
#exp(text,line): return string / none
#req(text,line,word): return string / list / none
#descri(text,line): return list / none
#jobinfo(initial_casenum,final_casenum): print all the cases. return none.

section II: gathering data
_________________________________________
section III: finding abnormal job case (e.g no requirements)

In [186]:
#find the job case information,initial_casenum<=final_casenum
def jobinfo(initial_casenum,final_casenum):
    word=["Requirement","requirement","REQUIREMENT","Requirements","requirements","Requirement:","requirement:","Requirements:","requirements:",
      "Responsibility","responsibility","RESPONSIBILIT","Responsibilities","responsibilities","Responsibility:","responsibility:","Responsibilities:","responsibilities:"]

    for i in range(initial_casenum,final_casenum+1):
        text,line,soup=read(i)

        print("job case:",i)
        print("date:",date(text))
        print("job title:",job(soup))
        print("company name:",company(soup))
        print("location:",location(soup))
        print("industry:",industry(text,line))
        print("yrs of exp:",exp(text,line),"\n")
        for j in word:
            print(j,"=>",req(text,line,j),"\n")
        print("______________________The end________________________")
    return

****************************************Please read from here****************************************************
In some cases we could not find any requirements.
job case 3874 is one of the example where there is no job requirement.

In [187]:
jobinfo(3874,3874)

job case: 3874
date: 08-Oct-21
job title: Associate Professor / Assistant Professor (Artificial Intelligence and Educational Technology)
company name: The Education University of Hong Kong
location: 
industry: Education
yrs of exp: None 

Requirement => None 

requirement => None 

REQUIREMENT => None 

Requirements => None 

requirements => None 

Requirement: => None 

requirement: => None 

Requirements: => None 

requirements: => None 

Responsibility => None 

responsibility => None 

RESPONSIBILIT => None 

Responsibilities => None 

responsibilities => None 

Responsibility: => None 

responsibility: => None 

Responsibilities: => None 

responsibilities: => None 

______________________The end________________________


In [231]:
#since in most of the job cases the key words could be "requirement:","requirements"...etc, we searched all of them.

#to make a dataframe showing the requirement of job cases
word=["Requirement","requirement","REQUIREMENT","Requirements","requirements","Requirement:","requirement:","Requirements:","requirements:",
      "Responsibility","responsibility","RESPONSIBILIT","Responsibilities","responsibilities","Responsibility:","responsibility:","Responsibilities:","responsibilities:"]
df = pd.DataFrame(columns=word,index=range(3872,4001))


for i in range(3872,4001):
    text,line,soup=read(i)

    for j in word:
        if (req(text,line,j)!=None)&(req(text,line,j)!=[]):
            df.loc[i,j] = 1 #if the requirement/responsibility is not empty,insert 1.

In [232]:
df
#if the requirement/responsibility is not empty,it shows 1.

,Requirement,requirement,REQUIREMENT,Requirements,requirements,Requirement:,requirement:,Requirements:,requirements:,Responsibility,responsibility,RESPONSIBILIT,Responsibilities,responsibilities,Responsibility:,responsibility:,Responsibilities:,responsibilities:
3872,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3873,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3875,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3876,1,1,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3997,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN
3998,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3999,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN


In [233]:
#Question: Which job case does not have any requirement or responsibility?
df["sum_req"] = df.notnull().iloc[:,:9].sum(axis=1) #it is the number of job requirements in the job case.
df["sum_res"] = df.notnull().iloc[:,9:18].sum(axis=1) #it is the number of job responsibilities in the job case.

In [234]:
df

,Requirement,requirement,REQUIREMENT,Requirements,requirements,Requirement:,requirement:,Requirements:,requirements:,Responsibility,responsibility,RESPONSIBILIT,Responsibilities,responsibilities,Responsibility:,responsibility:,Responsibilities:,responsibilities:,sum_req,sum_res
3872,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3873,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,3,2
3874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3875,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,5,2
3876,1,1,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,3,2
3997,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,5,2
3998,1,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,3,1
3999,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1,NaN,5,2


In [235]:
print(sum(df.sum_req==0)) #there are 23 job cases which have NO requirements
print(sum(df.sum_res==0)) #there are 39 job cases which have NO responsibilities.
print(len(df)) #I have only taken 129 samples of job cases for inspection.

23
39
129


In [236]:
#Let's find the case which have NO requirements and NO responsibilities.
abnormal=df[(df.sum_req==0) & (df.sum_res==0)]
print(len(abnormal)) #there are 16 job cases which have no req/res.
abnormal #it shows all. To find the information of the job case, use the function jobinfo(initial_casenum,final_casenum).

16


,Requirement,requirement,REQUIREMENT,Requirements,requirements,Requirement:,requirement:,Requirements:,requirements:,Responsibility,responsibility,RESPONSIBILIT,Responsibilities,responsibilities,Responsibility:,responsibility:,Responsibilities:,responsibilities:,sum_req,sum_res
3874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [237]:
#job case 3981 is another example of no requirement
jobinfo(3981,3981)
#After searching the job content on Jobsdb, we found that there is no requirement because they used the keyword "job description"

job case: 3981
date: 27-Oct-21
job title: IT Support
company name: Varadise Limited
location: Shatin Area
industry: Architecture/Building/Construction
yrs of exp: 2 years 

Requirement => None 

requirement => None 

REQUIREMENT => None 

Requirements => None 

requirements => None 

Requirement: => None 

requirement: => None 

Requirements: => None 

requirements: => None 

Responsibility => None 

responsibility => None 

RESPONSIBILIT => None 

Responsibilities => None 

responsibilities => None 

Responsibility: => None 

responsibility: => None 

Responsibilities: => None 

responsibilities: => None 

______________________The end________________________


In [238]:
#Which keywords are used the most?
df.notnull().sum()
#we use it for reference to decide which keywords should be found first.

Requirement           87
requirement           52
REQUIREMENT            4
Requirements          77
requirements          37
Requirement:           7
requirement:           0
Requirements:         53
requirements:          1
Responsibility         3
responsibility         7
RESPONSIBILIT          9
Responsibilities      72
responsibilities       9
Responsibility:        1
responsibility:        0
Responsibilities:     47
responsibilities:      4
sum_req              129
sum_res              129
dtype: int64

In [239]:
#What if we search Job Description?

print(abnormal.index)#abnormal is the dataframe showing all abnormal cases with no requirements/responsibilities.

for i in abnormal.index:
    text,line,soup=read(int(i))
    print("case number:",i)
    print("Any Job Description?=>",req(text,line,"Job Description"))
#If we search job description, we found the job description successfully.

Int64Index([3874, 3883, 3884, 3890, 3893, 3897, 3901, 3902, 3908, 3918, 3948,
            3951, 3953, 3968, 3981, 3986],
           dtype='int64')
case number: 3874
Any Job Description?=> ['(Ref: AssoProf/AsstProf/MIT/0921)', 'Department of Mathematics and Information Technology Faculty of Liberal Arts and Social Sciences', 'The Department is now seeking enthusiastic, qualified and highly dedicated person to take part in a number of teaching and research activities in artificial intelligence and educational technology. The appointee is expected to teach undergraduate and postgraduate courses, conduct research, supervise student projects and field experiences, and undertake administrative duties as assigned.', 'For information on the Department, please visit this website:', 'http://www.eduhk.hk/mit.', 'Salary will be commensurate with qualifications and experience.', 'Initial appointment will be made on a fixed-term contract. Fringe benefits include contract-end gratuity, leave, medical

case number: 3902
Any Job Description?=> ['What effort do you need to make?', 'Handle technical tickets of all Incident and Fault at Customer’s NOC.', 'Troubleshoot &amp; resolve the incidents and problem tickets within SLA', 'Perform Advanced Level Troubleshooting &amp; Analysis of complex problem', 'Interacts effectively with Cisco TAC, Vendor TAC and Engineering to solve complex problems.', 'Simulate the Customer Network architecture within Operations Lab to resolve complex problems and compatibility issues.', 'Collaborate with cross functional teams of Customer, Cisco teams to deliver improved Operations KPI', 'Collaborates with Vendor Engineering teams on complex product and software issues, driving resolution of specific bugs.', 'Share knowledge to the team member’s and participate in the various training programs.', 'Let’s check if you are…']
case number: 3908
Any Job Description?=> ['Your role:', 'Handle customer service hotlines and\xa0provide ease of contact service to custom

case number: 3968
Any Job Description?=> ['Your role', 'Implement new product initiatives\xa0aiming to deliver best features and usability in mobile payment services', 'Work closely with internal business stakeholders, IT and Development Team to define product feature and function specifications', 'Manage and coordinate with external partners on new feature implementation', 'Manage the product development lifecycle with multiple stakeholders from initiative to post-implementation', 'Prepare and review product specification and documentation', 'Ensure timely and quality project deliverables by defining and conducting comprehensive project &amp; UAT plan', 'To succeed in this role', 'Bachelor degree in Information Technology, Computer Science or related disciplines', '5 years’ relevant product development and management experience in a multi-national FinTech company or IT/ technology consulting firm', 'Solid experience in digital, e-Business, card scheme, mobile wallet and/ or payment re

In [240]:
#The work above tells us that we should search the job description.

#job case 3968 is another example with no requirement/responsibility.
text,line,soup=read(3968)
wordlocation(text,line,"Job Description",1,80)

['Your role',
 'Implement new product initiatives\xa0aiming to deliver best features and usability in mobile payment services',
 'Work closely with internal business stakeholders, IT and Development Team to define product feature and function specifications',
 'Manage and coordinate with external partners on new feature implementation',
 'Manage the product development lifecycle with multiple stakeholders from initiative to post-implementation',
 'Prepare and review product specification and documentation',
 'Ensure timely and quality project deliverables by defining and conducting comprehensive project &amp; UAT plan',
 'To succeed in this role',
 'Bachelor degree in Information Technology, Computer Science or related disciplines',
 '5 years’ relevant product development and management experience in a multi-national FinTech company or IT/ technology consulting firm',
 'Solid experience in digital, e-Business, card scheme, mobile wallet and/ or payment related projects with experience 

In [241]:
#i am trying to see if we can successfully find the job description of cases 3872 to 4000. (4001 is not included)
df['job description']=''
for i in range(3872,4001):
    text,line,soup=read(i)
    if (descri(text,line)!=[])|(descri(text,line)!=None):
        df.loc[i,'job description']=1

In [244]:
df.sum()

Requirement           87
requirement           52
REQUIREMENT            4
Requirements          77
requirements          37
Requirement:           7
requirement:           0
Requirements:         53
requirements:          1
Responsibility         3
responsibility         7
RESPONSIBILIT          9
Responsibilities      72
responsibilities       9
Responsibility:        1
responsibility:        0
Responsibilities:     47
responsibilities:      4
sum_req              318
sum_res              152
job description      129
dtype: object